In [1]:
from google.cloud import bigquery
from google.cloud import aiplatform
import bigframes.pandas as bpd
import pandas as pd
from vertexai.language_models._language_models import TextGenerationModel
from bigframes.ml.cluster import KMeans
from bigframes.ml.model_selection import train_test_split

In [2]:
project_id = 'qwiklabs-gcp-03-4138d4db502c'
dataset_name = "ecommerce"
model_name = "customer_segmentation_model"
table_name = "customer_stats"
location = "us-central1"
client = bigquery.Client(project=project_id)
aiplatform.init(project=project_id, location=location)

In [3]:
%%bigquery
CREATE OR REPLACE TABLE ecommerce.customer_stats AS
SELECT
  user_id,
  DATE_DIFF(CURRENT_DATE(), CAST(MAX(order_created_date) AS DATE), day) AS days_since_last_order, ---RECENCY
  COUNT(order_id) AS count_orders, --FREQUENCY
  AVG(sale_price) AS average_spend --MONETARY
  FROM (
      SELECT
        user_id,
        order_id,
        sale_price,
        created_at AS order_created_date
        FROM `bigquery-public-data.thelook_ecommerce.order_items`
        WHERE
        created_at
            BETWEEN '2022-01-01' AND '2023-01-01'
  )
GROUP BY user_id;

Query is running:   0%|          |

""


In [4]:
# prompt: Convert the table ecommerce.customer_stats to a BigQuery DataFrames dataframe and show the top 10 records

bqdf = client.query(f"SELECT * FROM `{project_id}.{dataset_name}.{table_name}`").to_dataframe()
bqdf.head(10)

,user_id,days_since_last_order,count_orders,average_spend
0,90202,768,1,128.000000
1,67148,768,1,16.790001
2,78660,768,1,199.990005
3,89949,768,1,119.000000
4,67859,768,1,24.000000
5,76745,768,1,49.500000
6,5563,768,1,11.350000
7,49304,768,1,39.990002
8,27585,768,1,36.000000
9,22481,768,1,18.000000


In [5]:
df = bpd.read_gbq(f"{project_id}.{dataset_name}.{table_name}")
df.head(10)

,user_id,days_since_last_order,count_orders,average_spend
0,68248,914,1,43.5
1,191,662,2,53.665001
2,12545,888,1,49.5
3,78334,724,1,69.5
4,56879,681,1,11.44
5,50069,665,5,60.648
6,35867,738,2,16.995
7,97526,819,1,39.950001
8,24232,710,3,17.216667
9,3527,793,3,28.613334


In [6]:
#prompt: 1. Split df into test and training data for a K-means clustering algorithm store these as df_test_ and df_train. 2. Create a K-means cluster model using bigframes.ml.cluster KMeans with 5 clusters. 3. Save the model to BigQuery in a model called ecommerce.model_name using the to_gbq method.

df_train, df_test = train_test_split(df, test_size=0.2,  random_state=42)
model = KMeans(n_clusters=5)
model.fit(df_train)
model.to_gbq(f"{project_id}.{dataset_name}.{model_name}")

KMeans(distance_type='EUCLIDEAN', init='KMEANS_PLUS_PLUS', n_clusters=5)

In [7]:
# prompt: 1. Call the K-means prediction model on the df dataframe, and store the results as predictions_df and show the first 10 records.

predictions_df = model.predict(df)
predictions_df.head(10)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,user_id,days_since_last_order,count_orders,average_spend
0,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.157802235076...",68248,914,1,43.5
1,3,"[{'CENTROID_ID': 3, 'DISTANCE': 0.913769464247...",191,662,2,53.665001
2,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.833178317412...",12545,888,1,49.5
3,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.968544516038...",78334,724,1,69.5
4,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.367844582889...",56879,681,1,11.44
5,5,"[{'CENTROID_ID': 5, 'DISTANCE': 2.723897017012...",50069,665,5,60.648
6,3,"[{'CENTROID_ID': 3, 'DISTANCE': 1.082671259621...",35867,738,2,16.995
7,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.077488634415...",97526,819,1,39.950001
8,3,"[{'CENTROID_ID': 3, 'DISTANCE': 1.609159963203...",24232,710,3,17.216667
9,3,"[{'CENTROID_ID': 3, 'DISTANCE': 2.027231394063...",3527,793,3,28.613334


In [10]:
#prompt: 1. Using predictions_df, and matplotlib, generate a scatterplot. 2. On the x-axis of the scatterplot, display days_since_last_order and on the y-axis, display average_spend from predictions_df. 3. Color by cluster. The chart should be titled "Attribute grouped by K-means cluster."


import matplotlib.pyplot as plt

plt.scatter(predictions_df['days_since_last_order'], predictions_df['average_spend'], c=predictions_df['cluster'])
plt.xlabel("days_since_last_order")
plt.ylabel("average_spend")
plt.title("Attribute grouped by K-means Cluster")
plt.show()


KeyError: 'cluster'

In [11]:
query = """
SELECT
 CONCAT('cluster ', CAST(centroid_id as STRING)) as centroid,
 average_spend,
 count_orders,
 days_since_last_order
FROM (
 SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
 FROM ML.CENTROIDS(MODEL `{0}.{1}`)
)
PIVOT (
 SUM(value)
 FOR feature IN ('average_spend',  'count_orders', 'days_since_last_order')
)
ORDER BY centroid_id
""".format(dataset_name, model_name)

df_centroid = client.query(query).to_dataframe()
df_centroid.head()

,centroid,average_spend,count_orders,days_since_last_order
0,cluster 1,43.94,1.36,799.47
1,cluster 2,665.02,1.13,734.34
2,cluster 3,45.19,1.47,668.55
3,cluster 4,136.75,1.41,797.89
4,cluster 5,56.27,2.30,625.61


In [12]:
df_query = client.query(query).to_dataframe()
df_query.to_string(header=False, index=False)

cluster_info = []
for i, row in df_query.iterrows():
 cluster_info.append("{0}, average spend ${2}, count of orders per person {1}, days since last order {3}"
  .format(row["centroid"], row["count_orders"], row["average_spend"], row["days_since_last_order"]) )

cluster_info = (str.join("\n", cluster_info))
print(cluster_info)

cluster 1, average spend $43.94, count of orders per person 1.36, days since last order 799.47
cluster 2, average spend $665.02, count of orders per person 1.13, days since last order 734.34
cluster 3, average spend $45.19, count of orders per person 1.47, days since last order 668.55
cluster 4, average spend $136.75, count of orders per person 1.41, days since last order 797.89
cluster 5, average spend $56.27, count of orders per person 2.3, days since last order 625.61


In [14]:
prompt = f"""
You're a creative brand strategist, given the following clusters, come up with \
creative brand persona, a catchy title, and next marketing action, \
explained step by step.

Clusters:
{cluster_info}

For each Cluster:
* Title:
* Persona:
* Next marketing step:
"""

In [15]:
#prompt:  Use the Vertex AI language_models API to call the PaLM2 text-bison model and generate a marketing campaign using the variable prompt. Use the following model settings: max_output_tokens=1024, temperature=0.4

model = TextGenerationModel.from_pretrained("text-bison")
response = model.predict(prompt, max_output_tokens=1024, temperature=0.4)
print(response.text)

 **Cluster 1**

**Title:** The Occasional Shoppers

**Persona:** These customers make infrequent purchases, with an average of 1.36 orders per person and a long time between orders (799.47 days). They tend to spend a moderate amount per order ($43.94), indicating that they may be looking for specific items or making occasional purchases.

**Next marketing step:**
- **Targeted email campaigns:** Send personalized emails to these customers highlighting new products or special offers that align with their previous purchases.
- **Remarketing ads:** Use remarketing ads to remind these customers about products they've viewed or added to their carts but didn't purchase.
- **Loyalty program:** Implement a loyalty program to reward these customers for their occasional purchases and encourage them to make more frequent visits.

**Cluster 2**

**Title:** The Big Spenders

**Persona:** These customers are characterized by their high average spend ($665.02) and relatively low order frequency (1.13 

In [16]:
# Delete customer_stats table

client.delete_table(f"{project_id}.{dataset_name}.{table_name}", not_found_ok=True)
print(f"Deleted table: {project_id}.{dataset_name}.{table_name}")


# Delete K-means model
client.delete_model(f"{project_id}.{dataset_name}.{model_name}", not_found_ok=True)
print(f"Deleted model: {project_id}.{dataset_name}.{model_name}")


Deleted table: qwiklabs-gcp-03-4138d4db502c.ecommerce.customer_stats
Deleted model: qwiklabs-gcp-03-4138d4db502c.ecommerce.customer_segmentation_model
